### OBSOLETE
A very early version of a notebook that shows how to preprocess a dataset.


In [ ]:
import os 
import cv2
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#DATA_FILE = "./data/Qinglian_20210420_MD-295_NB_at_I-695_Crash_ 02102021_1448_MDOT_SHA_CCTV.xlsx"
#VIDEO_FILE = "/data/MoveOver/Videos/Videos_20210213/MD-295 NB at I-695 - Crash - 02102021@1448 - MDOT SHA CCTV.mp4"

DATA_FILE = "./data_old/Qinglian_test_8_D8_30_PA_501_2021_03_02_FHWA.xlsx"
VIDEO_FILE = "./data_old/test_8_D8_30_PA_501_2021_03_02_FHWA.mp4.avi"

In [ ]:
COLORS = {
    'vehicle' : (255, 0, 0),
    'truck' : (0, 255, 0),
    'motorcycle' : (0, 0, 255),
    'other' : (255, 255, 255)
}

In [ ]:
def read_data(data_file):
    df = pd.read_excel(data_file)
    df.TimeStamp = pd.to_timedelta(df.TimeStamp)
    tmp = df.groupby('TimeStamp').size().sort_index().reset_index().drop(0, axis=1)
    tmp['frame'] = np.arange(len(tmp))
    df = df.merge(tmp, on='TimeStamp')
    return df

def set_first_frame(df, fps):
    first_time = df.TimeStamp.astype(int).min() / 1e9
    if first_time > 0:
        first_frame = np.round(first_time * fps)
        print ('First frame = {}'.format(first_frame))
        tmp = df.groupby('TimeStamp').size().sort_index().reset_index().drop(0, axis=1)
        tmp['frame'] = np.arange(len(tmp)) + first_frame
        df = df.drop('frame', axis=1).merge(tmp, on='TimeStamp')
    return df


df = read_data(DATA_FILE)
print ('{} frames and {} objects loaded.'.format(len(df.frame.unique()), len(df)))
df.head()


In [ ]:
df.type.value_counts()

In [ ]:
cap = cv2.VideoCapture(VIDEO_FILE) 
fps = cap.get(cv2.CAP_PROP_FPS)
print ('FPS:', fps)
df = set_first_frame(df, fps)
df.frame = df.frame.astype(int)
df.head()

In [ ]:
def add_rectangles(image, df):
    for index, row in df.iterrows():
        if row.type in COLORS:
            color = COLORS[row.type]
        else:
            color = COLORS['other']
        cv2.rectangle(image, (int (row.x), int(row.y)), (int (row.x + row.width), int (row.y + row.height)), color, 3)
    return image

def display_frame(cap, df, frameno = 1):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frameno)
    ret, image = cap.read()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = add_rectangles(image, df[df.frame == frameno])

    plt.figure(figsize=(12, 12))
    plt.imshow (image)
    plt.title('Frame no. {}'.format(frameno))
    return image

def display_frames(cap, df, frame_start = None, frame_end = None):
    if frame_start is None:
        frame_start = df.frame.min()
    if frame_end is None:
        frame_end = df.frame.max()
    frameno = frame_start
    cap.set(cv2.CAP_PROP_POS_FRAMES, frameno)
    while (frameno < frame_end):
        ret, image = cap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = add_rectangles(image, df[df.frame == frameno])

        plt.figure(figsize=(12, 12))
        plt.imshow (image)
        plt.title('Frame no. {}'.format(frameno))
        plt.show()
        frameno += 1
        print (frameno)
    


### How to display one frame

In [ ]:
image1 = display_frame(cap, df, df.frame.min())

### How to display all frames

In [ ]:
display_frames(cap, df)